In [30]:
import re
import string
import itertools as itools
import heapq as hq
from collections import defaultdict

class Maze:
    
    def __init__(self, dictionary):
        
        self.maze = dictionary
        self.adjacent = [1, -1, 1j, -1j]
        self.worldString = ''.join(set(self.maze[x] for x in self.maze))
        
        self.doors = defaultdict(list)
    
        return

    # Wrapper to access a given coordinate location. Returns 'None' if location is out of bound
    def access_loc(self, coord):
        
        found = None
        try:
            found = self.maze[coord]
        
        except:
            pass
        
        return found
        
    # Returns a dictionary of the neighboring points in the graph as {coordinates: element, ...}
    # kwarg 'select' is a regular expression that can be used to limit the search to
    # a given string that matches the RE. For example, we may exclude walls
    # by passing select="^#". If not specified, all neighbors are returned.
    # ADDED 14/2/2021 --> 'item_to_count' (string): if specified, the method
    # returns how many of the neighbors of 'point' match the 'item_to_count'
    # ADDED 14/2/2021 --> 'check_on_border' (bool) : if true, the method returns a boolean value
    # that asserts if the point is on the edge of the graph.
    def near_neighbors(self, point, select=None, item_to_count=None, check_on_border=False):
        
        neighbor_dict = dict()
        item_count = 0
        
        for adj in self.adjacent:
            
            neighbor = point+adj
            obj = self.access_loc(neighbor)
            
            if obj is not None:
                
                if (select is None) or (obj in re.findall(select, self.worldString)):
                    neighbor_dict[neighbor] = obj
                    item_count += 1 if obj==item_to_count else 0
                    
            elif check_on_border==True:
                return True
         
        if check_on_border==True:
            return False
        
        return neighbor_dict if item_to_count==None else item_count
        

    # This is not necessary, just a funny old thing that could be useful.
    # Fills all the dead ends of the maze with '#', effectively creating a new maze
    # which does not have dead ends. Could help in solving mazes which have a
    # unique solution.
    def fill_dead_ends(self):
        
        complete = False
        
        # Iterate until there are no more dead ends
        while complete != True:
            
            complete = True
            
            # Scan the maze
            for key,val in self.maze.items():
                
                # Dead ends are '.' sorrounded by 3 '#', so
                # first of all we only want to operate if we are ona '.'
                if val != '.':
                    continue
                
                # Then check if the point has3 neighboring '#',
                # if yes replace the '.' with a '#'
                if self.near_neighbors(key, item_to_count='#') >= 3:
                    self.maze[key] = '#'
                    complete = False
                    
        return
    
    
    # [Day 20 specific] Function that helps create a dictionary for the problem
    # In particular, the dictionary is loaded in a hard-to-use pattern in which
    # couples of letters are separate dictionary entries (for example AA is stored as
    # { ..., <coordinates>: A, <coordinates>: A, ...}), when it would be way more handy 
    # to have them as atomic entites (as in {..., <coordinates>: AA, ...}). This
    # function refactors the dictionary according to this principle.
    def refactor(self):
        
        # Cycle over dictionary entries
        for key, val in self.maze.items():
            
            # If the current entry is a letter AND either
            # - has 3 neighboring white spaces (inside of the torus)
            # - is on the edge of the maze (outside of the torus)
            on_border = self.near_neighbors(key, check_on_border=True)
            if val in string.ascii_uppercase and (self.near_neighbors(key, item_to_count=' ') == 3 or on_border):
                
                # Take the neighboring letter (may be top, bottom, right or left):
                # this is the letter which is adjacent to the maze
                nKey, nVal = tuple(*self.near_neighbors(key, select="[A-Z]").items())
                # Concat the neighboring string with the current string 
                # There are two cases we must distinguish if we want
                # to concatenate consitently:
                # - the letter near the maze is on the left or above our letter, as in   #  #        #.#
                if (nKey.real - key.real)+(key.imag - nKey.imag)>0:                      #  .AA  or   A
                    self.maze[nKey] = val + nVal                                         #  #         A
                # - the letter near the maze is on the right or below our letter
                else:
                    self.maze[nKey] = nVal + val
                
                # Replace the letter in the old position with a white space or a star if 
                # it is on the border (useful for later to distinguish inside from outside)
                self.maze[key] = " " if not on_border else "*"
                
            
        return
                    
    
    # Initialize the list of doors. 
    # For meaningful result, call only after refactor() method
    def init_door_list(self):       
        
        for k,v in self.maze.items():
            if set(string.ascii_uppercase) > set(v):
                self.doors[v].append(k)
        
        return
    
    # Given a door at one position, find the twin door and return its position
    def jump(self, door_position, door_name):
        
        # Get the position of the twin door
        new_pos = self.doors[door_name][0] if self.doors[door_name][0] != door_position else self.doors[door_name][1] 
        
        # Also move inside the maze: look for the '.' near the door 
        # (there is only 1) and return its position 
        x, point = tuple(*self.near_neighbors(new_pos, select="\.").items())
        
        return x
    
     
    # Returns a string representing the maze
    # The loop over dictionary entries
    # is done in the order in which the elements were loaded 
    # in the dictionary, that is why this simple approach is doable.
    # Also, the strange looking 'if' conditions inside the loop are 
    # just to format the output in the nicest way, 
    def stringify(self):
        
        maze_str = ''
  
        for x,y in self.maze.items():
            
            if x.real == 0:
                maze_str += '\n' if len(self.maze[x+1])>1 else '\n '
            
            if len(y) > 1:
                maze_str += '\b' if (maze_str[-1] != "." and maze_str[-1] != "*" )  else ""
            
            maze_str += y
            
        return maze_str
    
    
    
    # Calculates the (shortest) distance between two doors in the maze
    def distance(self, start_pos, end_name):
        
        if self.maze[start_pos] == end_name:
            return 0
        
        # Initialize list of heads, visited positions
        heads = [start_pos]
        visited = [start_pos]
        
        # d starts at -1 because the step from the node of the door 
        # to inside the maze is no counted
        d = -1
        
        while heads != []:
            
            d += 1            
            new_heads = []

            for head in heads:
            
                # Get list of neighboring sites. Get all of them (don't use select)
                # since it would never match the doors
                neighbors = self.near_neighbors(head)
                
                for x,val in neighbors.items():
                    # Exclude walls (#),empty spaces ( ) and already visited positions
                    if val in "#* " or x in visited:
                        continue
                   
                    # If we've found b, exit. We must subtract the last increment to the distance 
                    elif val == end_name:
                        return d-1
                    
                    # If the neighbor is a point, we will move there
                    elif val == ".":
                        new_heads.append(x)
                        visited.append(x)
                        
                    # If the neighbor is a door, jump to the twin door via the jump function
                    elif val in self.doors:
                        new_heads.append(self.jump(x, val))
                        visited.append(x)

            # Replace the heads with the new heads
            heads = new_heads[:]
        
        return None
        
#********************************************************************************#        
        
    # Calculates the (shortest) distance between two doors in the recursive maze.
    # It is a 'recursification' of the previous 'distance()' function (specific to our 
    # particular maze), but I think, since my implementations are already messy,
    # it is a good idea to keep them separate instead of making a big mess in one function.
    # However, the principle by which the maze is explored (flood fill) is the same, 
    # so understanding the first one could help in understanding this one.
    #
    # The rsearch function takes different inputs, let's see what they are:
    # 1 - start_pos: coordinate of starting point
    # 2 - start_name: name of the starting point (the name of the door)
    # 3 - end_name: the door that we are looking for
    # 4 - distance: the distance of the path up to the current call
    # 5 - depth: from 0 up, the depth of the current call inside the recursive structure
    # 6 - sequence_id: a string containing the history of the path given by the
    #     series of doors that it went through, each followed by the corresponding
    #     depth. For example, the sequence: AA0BB1CC2 means that the path started
    #     at door AA at level 0, then went to level 1 via BB and then level 2 via CC.
    #     This is important for avoiding getting stuck in loops, see below.
    def rsearch(self, start_pos, end_name, distance, depth, sequence_id):
                
        # Initialize list heads
        heads = [start_pos]
        # Visited positions
        visited = [start_pos]
        # End of the sequence ID (has the form "LLNNN.." with L=letter, N=number)
        seq_id_end = re.search("[A-Z]+[0-9]+$", sequence_id)[0]
        # List of argument of the recursive calls
        l = []
        
        d = distance
        
        while heads != []:
            
            d += 1            
            new_heads = []

            for head in heads:
            
                # Get list of neighboring sites. We don't use 'select'
                # since it would never match the doors
                neighbors = self.near_neighbors(head)
                
                for x,val in neighbors.items():
                    
                    # Exclude everything that we are not interested in, namely:
                    # walls, spaces, stars, the entrance AA and finally the last visited door + already visited positions
                    if val in "#* AA"+seq_id_end[0:2] or x in visited:
                        continue
                     
                    # If we found a door, we may face different cases
                    if val in self.doors:
                        
                        # If we found the end and the depth is 0, we have finished!
                        # Return the distance and the sequence_id of the path
                        if val == end_name and depth == 0:
                            return d-1, sequence_id+end_name
                        
                        # Else if we found the end but we are not at depth 0, ignore
                        elif val == end_name and depth != 0:
                            continue
                    
                        # This is the first important detail. We don't want to get stucked
                        # in loops, so we use the sequence_id of the current path: we want to
                        # check that the string-pattern formed by:
                        # $last door in seq_id + depth + door we just found 
                        # is not already present inside the sequence_id itself.
                        # For example, take "AA0BB1CC2BB1" and suppose we just found again door CC.
                        # Clearly that would bring us in a loop, as "BB1CC2BB1CC2..." and so on (
                        # implicitly we are assuming that going to CC would bring us again to level 2,
                        # i.e. that door BB is connected to only *one* door CC, the one on the inside
                        # which bring us deeper in the maze).
                        # The minimum amount of information we need to avoid this situation is, 
                        # as explained above, the last door in the sequence ("BB" in our case), 
                        # followed by the current depth (1), followed by the candidate door that
                        # we would like to go to ("CC"). The pattern "BB1CC" is already found in the
                        # sequence_id, so this road is discarted.
                        if seq_id_end+val in sequence_id:
                            continue
                        
                        # If the door is on the outside, prepare to go up a level (delay actual recursive call**)
                        # This is also the reason why we added the '*' inside the 'refactor()' : doors
                        # on the outside have a neighboring '*', while doors on the inside have
                        # only white spaces (3 in total)
                        elif self.near_neighbors(x, item_to_count="*") > 0 and depth != 0:
                            l.append((self.jump(x, val), end_name, d, depth-1, sequence_id+val+str(depth-1)))
                        # If it is on the inside, prepare to go down (delay actual recursive call**)
                        elif self.near_neighbors(x, item_to_count=" ") == 3 :
                            l.append((self.jump(x, val), end_name, d, depth+1, sequence_id+val+str(depth+1)))
                            
                    # If the neighbor is a point, move there, and add it 
                    elif val == ".":
                        new_heads.append(x)
                        visited.append(x)
                        
            # Replace the heads with the new heads
            heads = new_heads
        
        # **
        # We delay the call to the recursive function for one fundamental reason: 
        # we want to insert a "bias" so that the recursive algorithm always tries 
        # to climb back up the recursive tree (go to depth = 0), before 
        # digging further into it. If we do not insert this bias we might get stucked
        # in a situation where the algorithms continues to go down in depths and misses the exit.
        # TO implement this, we sort our list of function calls according to the value of
        # the 4th argument, i.e. the 'depth'.
        l.sort(key=lambda x: x[3])
        
        # Now we can finally launch the recursive call
        for args in l:
            # Call the function
            result = self.rsearch(*args)
            # To exit the recursive tree
            if result is not None:
                return result

        return None
        
        
        
  

In [38]:
x = 0
y = 0
loadMaze = dict()

# Load maze from file
with open("input.txt", "r") as infile:
    
    for line in infile:
        for obj in list(line.replace("\n", "")):
            loadMaze[x+y] = obj
            x += 1
            
        x = 0
        y -= 1j
        

In [39]:
# Instance of our Maze object, passing the maze to the constructor
maze = Maze(loadMaze)

# Print maze
#print(maze.stringify())

# Refactoring procedure on the maze, brings to useful form
maze.refactor()

# Isolate door list
maze.init_door_list()

# Optional fill of dead ends. If nothing, speeds up execution
maze.fill_dead_ends()
# Uncomment to see the effect of the 'fill_dead_ends()' function
#print(maze.stringify())

In [40]:
#### PART 1
print("The shortest path between door 'AA' and door 'ZZ' is: ", maze.distance("AA", "ZZ"))

The shortest path between door 'AA' and door 'ZZ' is:  442


In [42]:
#### PART 2
start = maze.doors["AA"][0]
dist, history = maze.rsearch(start, "ZZ", -1, 0, "AA0")
print("The distance between AA and ZZ in the recursive maze is: ", dist)
print("The maximum depth we reach in exploring the recursive maze is", max([int(x) for x in re.findall("[0-9]+", history)]))
print("\n\nHere is the history of the path:")
for door,lev in zip(re.findall("[A-Z]+", history), re.findall("[0-9]+", history)):
    print("--> entering level ", lev, " via door ", door)


The distance between AA and ZZ in the recursive maze is:  5208
The maximum depth we reach in exploring the recursive maze is 25


Here is the history of the path:
--> entering level  0  via door  AA
--> entering level  1  via door  IG
--> entering level  2  via door  PZ
--> entering level  3  via door  RF
--> entering level  4  via door  ER
--> entering level  5  via door  EG
--> entering level  6  via door  AU
--> entering level  7  via door  ZH
--> entering level  8  via door  GR
--> entering level  9  via door  OK
--> entering level  10  via door  LQ
--> entering level  11  via door  UU
--> entering level  12  via door  UA
--> entering level  13  via door  RU
--> entering level  14  via door  KP
--> entering level  15  via door  RK
--> entering level  16  via door  LP
--> entering level  17  via door  BH
--> entering level  18  via door  LW
--> entering level  19  via door  ON
--> entering level  20  via door  CU
--> entering level  21  via door  BT
--> entering level  22  via door 

In [25]:
string = "AA"
print(re.search("[A-Z]+[0-9]+$", string)[0])

TypeError: 'NoneType' object is not subscriptable